# Méthodes ensemblistes ☕️☕️

**<span style='color:blue'> Objectifs de la séquence</span>** 
* Être sensibilisé&nbsp;:
    * aux gains de performances possibles via ce genre de méthodes,
    * à l'effet d'un ensemble de modèles sur la variance de notre prédiction.
* Être capable&nbsp;:
    * de produire des ensembles de modèles,
    * de se servir de $\texttt{sklearn}$ pour faire des ensembles,
    * et utiliser les forêts aléatoires, les boostings, etc.
    


 ----

## Introduction

Soit $\mathcal{X}$ notre espace d'entrée et $\mathcal{Y}$ notre espace de sortie. Soit $X, Y$ deux variables aléatoires sur $\mathcal{X}$ et $\mathcal{Y}$ et soit $\mathbb{P}$ leur mesure jointe. Notre objectif est de trouver une application $h:\mathcal{X}\mapsto\mathcal{Y}$ qui minimise une certaine erreur qu'on notera $L$. N'ayant pas accès aux variables aléatoires $X$ et $Y$, nous collectons un jeu de données $S_n=\{(X_i, Y_i)\}_{i\leq n}\sim\mathbb{P}^n$ (par exemple en récupérant des images sur internet) et nous construisons un risque empirique :

$$L_n(h)=\frac{1}{n}\sum_{i}\ell(h(X_i), Y_i),$$

où $\ell$ définit une erreur élémentaire (i.e. pour une unique prédiction). Comme nous l'avons vu, ce risque empirique $L_n$ est un estimateur de $L$. La fonction $h$ est ainsi construite en utilisant le jeu de données $S_n$.



## I. L'approche naïve

L'approche la plus simple lorsqu'on cherche à combiner plusieurs modèles consiste à moyenner leur prédiction. Dans le cas de la régression, la prédiction est la moyenne traditionnelle. Dans le cas de la classification, on utilisera un vote à la majorité simple.

---

Soit $\mathcal{Y}\subseteq\mathbb{R}$ l'espace des labels et $\mathcal{X}$ l'espace de nos données d'entrée. Considérons une famille de prédicteurs $\{y_j\}_{j\leq m}$, l'agrégation de ces derniers se fait de la manière la plus naïve qui soit (comme nous l'avons vu au-dessus)&nbsp;:

$$\hat{y}=\frac{1}{m}\sum_j y_j(x).$$

Supposons que nous ayons pour un prédicteur donné $\hat{y}_{ij} = y(x_j)+\epsilon_{ij}$ où la prédiction se fait à un bruit près qui dépend de l'échantillon mais aussi du prédicteur. Nous avons évidemment la relation suivante&nbsp;:

$$\mathbb{E}\big[(\hat{y}_j(X)-y(X))^2\big]=\mathbb{E}\big[\epsilon_{j}^2\big],$$

où $\epsilon_j$ dépend de $X$. De manière similaire, si cette fois-ci nous considérons l'agrégation de nos prédicteurs, nous avons&nbsp;:

$$\mathbb{E}\big[(y(X)-\frac{1}{m}\sum_j \hat{y_j}(X))^2\big]=\mathbb{E}\big[(\frac{1}{m}\sum_j\epsilon_{j})^2\big].$$

Si on suppose (ce qui n'est pas vraiment le cas en pratique)&nbsp;:

$$\begin{aligned}
\mathbb{E}\big[\epsilon_j\big]&=0\\
\mathbb{E}\big[\epsilon_j\epsilon_k\big]&=0,\ j\neq k,
\end{aligned}$$
nous avons&nbsp;:

$$\mathbb{E}\big[(\frac{1}{m}\sum_j\epsilon_{j})^2\big]=\frac{1}{m^2}\sum_i\mathbb{E}\big[\epsilon^2\big]=\frac{1}{m}\mathbb{E}_{\bar{\epsilon}},$$

où $\mathbb{E}_{\bar{\epsilon}}$ indique l'erreur moyenne de nos prédicteurs. L'erreur diminue au fur et à mesure que l'on ajoute des prédicteurs.

---

In [ ]:
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt

data = load_digits()

_, axes = plt.subplots(nrows=1, ncols=4, figsize=(10, 3))
for ax, image, label in zip(axes, data.images, data.target):
    ax.set_axis_off()
    ax.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    ax.set_title('Training: %i' % label)

In [ ]:
from sklearn.model_selection import train_test_split
X = data.data / data.data.max()

X_train, X_test, y_train, y_test = train_test_split(X, data.target, test_size=0.75)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from scipy import stats

**<span style='color:blue'> Exercice</span>** 
**Complétez la méthode $\texttt{predict}$ de la classe suivante afin de faire un vote à la majorité simple pour combiner plusieurs modèles.**



 ----

In [ ]:
class MajorityVoting(object):
    def __init__(self, *models):
        self.models = models
    
    def fit(self, X, y):
        for m in self.models:
            m.fit(X, y)

    def predict(self, X):
        ####### Complete this part ######## or die ####################
        ...
        ...
        ...
        return ...
        ###############################################################
    
    def score(self, X, y):
        scores = {'model': [], 'vote': None}
        for m in self.models:
            scores['model'].append(m.score(X, y))
        scores['vote'] = (self.predict(X) == y).astype(int).sum()/len(X)
        return scores
            
model = MajorityVoting(
    LogisticRegression(max_iter=5000), 
    DecisionTreeClassifier(),
    KNeighborsClassifier()
)
model.fit(X_train, y_train)

scores = model.score(X_test, y_test)

print('Nos modèles atteignent:', scores['model'])
print('Notre agrégation atteint:', scores['vote'])


On peut imaginer que les modèles se trompent sur les mêmes images et non de manière aléatoire. C'est par exemple le cas si les modèles se trompent lorsqu'un chiffre est mal dessiné et ressemble à un autre chiffre : tous les modèles vont faire la même erreur. Dans ces cas de figure, le meilleur modèle est le meilleur modèle et non le vote. À l'inverse, si nos différents modèles ont tendance à faire des erreurs différentes, alors l'agrégation permettra de gagner en score.

Considérons maintenant le jeu de données suivant.

In [ ]:
from sklearn.datasets import load_boston

data = load_boston()

In [ ]:
from sklearn.model_selection import train_test_split
X = data.data/data.data.max(axis=0)
X_train, X_test, y_train, y_test = train_test_split(X, data.target, test_size=0.75)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
from scipy import stats

from sklearn.metrics import mean_squared_error

**<span style='color:blue'> Exercice</span>** 
**Complétez la méthode $\texttt{predict}$ de la classe suivante afin de faire une moyenne pour combiner plusieurs modèles.**



 ----

In [ ]:
class AverageVoting(object):
    def __init__(self, *models):
        self.models = models
    
    def fit(self, X, y):
        for m in self.models:
            m.fit(X, y)

    def predict(self, X):
        ####### Complete this part ######## or die ####################
        ...
        ...
        ...
        return ...
        ###############################################################
    
    def score(self, X, y):
        scores = {'model': [], 'vote': None}
        for m in self.models:
            scores['model'].append(mean_squared_error(y, m.predict(X)))
        scores['vote'] = mean_squared_error(y, self.predict(X))
        return scores
            
model = AverageVoting(
    LinearRegression(), 
    DecisionTreeRegressor(),
    KNeighborsRegressor()
)
model.fit(X_train, y_train)

scores = model.score(X_test, y_test)

print('Nos modèles atteignent:', scores['model'])
print('Notre agrégation atteint:', scores['vote'])


On observe cette fois-ci un gain clair de performances. Les différents modèles doivent se tromper d'une manière au moins partiellement aléatoire et répartie autour de la moyenne. L'agrégation rend ce résultat plus stable.

Ces manières d'agréger des votes est effectives mais restent "naïve". Nous allons voir que nous pouvons formaliser tout cela dans le cadre du *framework* probabiliste.

## II. Bayesian Model Averaging

Notons $z_i=(x_i, y_i)$ et une famille de $M$ modèles probabilistes (e.g. régression logistique) où chaque modèle est noté $\mathcal{M}_j$. Notons :

$$p(y_i|x_i, \mathcal{M}_j),$$

la densité d'un point de $\mathcal{Y}$ relativement au modèle $\mathcal{M}_j$ et à une observation $x_i\in\mathcal{X}$. Notre objectif est de déterminer dans un premier temps la "qualité" d'un modèle en tenant compte de notre point de vu *a priori* ainsi que des données que nous avons pu observer $S_n$. Notons ainsi $p(\mathcal{M}_j)$ notre probabilité *a priori* sur le modèle $\mathcal{M}_j$. Celle-ci peut favoriser certaines solutions parcimonieuses ou bien être uniforme et ne favoriser aucun modèle. En appliquant la règle de Bayes, nous obtenons :

$$p(\mathcal{M}_j|X, \boldsymbol{y})=\frac{p(\boldsymbol{y}|X, \mathcal{M}_j)p(\mathcal{M}_j)}{p(\boldsymbol{y}| X)},$$

où $S_n=(\boldsymbol{y}, X)$ et où nous avons :

$$p(\boldsymbol{y}|X, \mathcal{M}_j)=\prod_i p(y_i|x_i, \mathcal{M}_j),$$

ainsi que :

$$p(\boldsymbol{y}| X)=\sum_j p(\boldsymbol{y}|X, \mathcal{M}_j)p(\mathcal{M}_j).$$

Nous avons bien ici toutes les informations nous permettant d'évaluer de manière Bayésienne la qualité de nos différents modèles. Cependant, en pratique, les variations pourraient très bien dépendre d'un tirage particulier de nos données. L'idée derrière le *Bayesian Model Averaging* consiste à considérer TOUS les modèles mais à les pondérer par leur qualité. Cela ne se fait pas au doigt mouillé, mais en utilisant encore une fois le *framework* probabiliste :

$$p(y_\text{new}|x_\text{new}, S_n)=\sum_j p(y_\text{new}|x_\text{new}, \mathcal{M}_j, S_n)p(\mathcal{M}_j|S_n).$$


**Application a une famille de régression logistique.**

Dans le cas de la régression logistique, nous avons la vraisemblance suivante :

$$\mathcal{L}(\theta_j)=\prod_i p(y_i|x_i,\theta_j)$$

où

$$p(y_i|x_i, \theta_j)=\sigma_j(x_i)^y_i(1-\sigma_j(x_i))^{1-y_i}$$

et

$$\sigma_j(x)=(1+e^{-\langle\theta_j,x\rangle})^{-1}.$$

Ainsi, la *posterior* de notre modèle $\theta_j$ après avoir observé notre jeu de données est donnée par :

$$p(\theta_j|S_n)\propto \prod_i p(y_i|x_i,\theta_j)p(\theta_j),$$

où le symbole $\propto$ indique la proportionnalité.

Nous allons construire dans cette exercice un modèle de classification d'images de chiffres à partir de régressions logistiques. La particularité sera que chaque régression logistique ne verra qu'une partie de l'image tirée aléatoirement.

In [ ]:
from sklearn.datasets import load_digits
data = load_digits()

Affichons le jeu de données ainsi qu'un exemple de ce que pourrait voir un modèle.

In [ ]:
import matplotlib.pyplot as plt

_, axes = plt.subplots(nrows=1, ncols=4, figsize=(10, 3))
for ax, image, label in zip(axes, data.images, data.target):
    ax.set_axis_off()
    ax.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    ax.set_title('Training: %i' % label)

mask = np.random.binomial(n=1, p=0.4, size=data.images[0].shape).astype(bool)

_, axes = plt.subplots(nrows=1, ncols=4, figsize=(10, 3))
for ax, image, label in zip(axes, data.images, data.target):
    ax.set_axis_off()
    ax.imshow(
        np.ma.masked_array(image, mask=mask), 
        cmap=plt.cm.gray_r, 
        interpolation='nearest'
    )
    ax.set_title('Training: %i' % label)

Construisons notre jeu d'apprentissage ainsi que notre jeu de test.

In [ ]:
from sklearn.model_selection import train_test_split
X = data.data / data.data.max()

X_train, X_test, y_train, y_test = train_test_split(X, data.target, test_size=0.75)

**<span style='color:blue'> Exercice</span>** 
**Complétez les méthodes $\texttt{posterior_}$, $\texttt{fit}$ et $\texttt{predict}$ afin de reproduire le *framework* que nous avons introduit au-dessus.**



 ----

In [ ]:
from sklearn.linear_model import LogisticRegression
import numpy as np
from scipy.special import logsumexp

In [ ]:
class BayesianLogisticAveraging(object):
    def __init__(self, dim=64, p=0.5, nb_models=50, max_iter=5000):
        self.mask = np.random.binomial(n=1, p=p, size=(nb_models, dim)).astype(bool)
        self.nb_models = nb_models
        self.models = []
        self.posterior = []
        self.prior = 1./nb_models
        self.max_iter = max_iter
        
    def posterior_(self, i, X, y):
        ####### Complete this part ######## or die ####################
        ...
        ...

        return ...
        ###############################################################
        
        
    def fit(self, X_train, y_train):
        ####### Complete this part ######## or die ####################
        ...
        ...
        ...
        ...
        ###############################################################
        
    def individual_scores(self, X, y):
        scores = []
        for _ in range(len(self.models)):
            scores.append(self.models[_].score(X[:, self.mask[_]], y))
        print(np.max(scores))
            
    def predict(self, X):
        ####### Complete this part ######## or die ####################
        ...
        ...
        ...
        ...
        return ...
        ###############################################################
        
    def score(self, X, y):
        scores = {'model': [], 'vote': None}
        for i, m in enumerate(self.models):
            scores['model'].append(m.score(X[:, self.mask[i]], y))
        scores['vote'] = (self.predict(X) == y).astype(int).sum()/len(X)
        return scores

model = BayesianLogisticAveraging()
model.fit(X_train, y_train)

scores = model.score(X_test, y_test)

print('Nos modèles atteignent:\n -', '\n - '.join([str(i) for i in scores['model']]))
print('Notre agrégation atteint:', scores['vote'])



De la même manière que précédemment un gain est observé si les erreurs sont indépendantes. De plus, il est important d'avoir une bonne estimation des probabilités conditionnelles afin que la vraisemblance soit un bon indicateur de la qualité de notre modèle. Une stratégie alternative, souvent utilisée en *deep learning* consiste pour chaque prédiction à favoriser le modèle dont les scores de prédictions sont les plus élevés.

**<span style='color:blue'> Exercice</span>** 
**Complétez la méthodes $\texttt{predict}$ afin de que la prédiction choisie soit celle du modèle ayant le score de prédiction le plus élevé.**



 ----

In [ ]:
class LogisticMaxPooling(object):
    def __init__(self, dim=64, p=0.5, nb_models=50, max_iter=5000):
        self.mask = np.random.binomial(n=1, p=p, size=(nb_models, dim)).astype(bool)
        self.nb_models = nb_models
        self.models = []
        self.max_iter = max_iter
        
        
    def fit(self, X_train, y_train):
        for _ in range(self.nb_models):
            model = LogisticRegression(max_iter=self.max_iter)
            X = X_train[:, self.mask[_]]
            model.fit(X, y_train)
            self.models.append(model)
            
    def individual_scores(self, X, y):
        scores = []
        for _ in range(len(self.models)):
            scores.append(self.models[_].score(X[:, self.mask[_]], y))
        print('Best model:', np.max(scores), ':: Worst model:', np.min(scores))
            
    def predict(self, X):
        ####### Complete this part ######## or die ####################
        ...
        ...
        ...
        return ...
        ###############################################################
        
    def score(self, X, y):
        scores = {'model': [], 'vote': None}
        for i, m in enumerate(self.models):
            scores['model'].append(m.score(X[:, self.mask[i]], y))
        scores['vote'] = (self.predict(X) == y).astype(int).sum()/len(X)
        return scores

model = LogisticMaxPooling()
model.fit(X_train, y_train)
model.individual_scores(X_test, y_test)
model.predict(X_test)

scores = model.score(X_test, y_test)

print('Nos modèles atteignent:\n -', '\n - '.join([str(i) for i in scores['model']]))
print('Notre agrégation atteint:', scores['vote'])


Le principe du *Bayesian Model Averaging* se généralise bien sûr lorsqu'on a une quantité infinie de modèles et la *posterior* prédictive s'écrit :

$$p(y|x_\text{new}, S_n)=\int p(y|x_\text{new}, \mathcal{M}, S_n)p(\mathcal{M}|S_n)dM.$$

Cette stratégie demande un effort analytique cependant beaucoup plus intense. La séquence [Bayésienne linear regression](./2_bayesian_linear_regression.ipynb) illustrera notamment cette idée dans le cas de la régression linéaire et pointera du doigt son lien avec la notion de régularisation.


## III. Bagging


La difficulté de considérer plusieurs modèles est souvent que pour une classe de modèles donnée, un même jeu d'apprentissage implique une même solution et combiner plusieurs fois la même chose ne sert à rien. L'idée du bagging est d'agréger plusieurs modèles en trouvant justement une stratégie pour créer de la variabilité entre ces derniers. L'agrégation se fait de la manière la plus naïve possible en moyennant les prédictions comme indiqué au-dessus.

La stratégie utilisée pour créer de la variabilité est de passer par un jeu de données *bootstrap*. Soit $S_n=\{(X_i, Y_i)\}_{i\leq n}$ un jeu de données de taille $n$. L'idée va être de construire $m$ nouveaux jeux de données (pour chacun de nos modèles) à partir de $S_n$ en tirant $n$ points aléatoirement dans $S_n$ **avec** remise. Ainsi, certains points apparaîtront en double, et d'autres seront absents.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

Considérons la fonction suivante&nbsp;:

In [ ]:
def f(x):
    x = x.ravel()
    return np.exp(-(x-5) ** 2) + 2 * np.exp(-(x - 7) ** 2) + np.exp(-(x + 2) ** 2)

In [ ]:
def generate(n_samples, noise=0.1, n_repeat=1):
    X = np.random.rand(n_samples) * 10
    X = np.sort(X)

    if n_repeat == 1:
        y = f(X) + np.random.normal(0.0, noise, n_samples)
    else:
        y = np.zeros((n_samples, n_repeat))

        for i in range(n_repeat):
            y[:, i] = f(X) + np.random.normal(0.0, noise, n_samples)

    X = X.reshape((n_samples, 1))

    return X, y


X_train = []
y_train = []

n_repeat=50

X_test, y_test = generate(n_samples=1000, n_repeat=n_repeat)

for i in range(n_repeat):
    X, y = generate(n_samples=50)
    X_train.append(X)
    y_train.append(y)

Nous avons vu dans la séquence sur la régression linéaire que nous pouvions décomposer notre erreur entre une composante de biais et une composante de variance&nbsp;:

$$\begin{aligned}
\mathbb{E}\big[(y-\hat{f}(x))^2\big]=\sigma^2+\text{Var}\big(\hat{f}\big)+\text{bias}(\hat{f})^2.\end{aligned}$$

L'objectif ci-dessous va être de comparer les différentes briques de notre erreur. Pour cela, nous devons trouver une stratégie pour estimer chacune de ces quantités.

**<span style='color:blue'> Question</span>** 
**À votre avis, comment estimer empiriquement l'erreur totale ainsi que chacune des briques de l'erreur totale (en utilisant les jeux de données créés au-dessus) ?**



 ----

**<span style='color:blue'> Exercice</span>** 
**Complétez le code suivant afin d'entraîner un arbre et un bagging construit à partir d'arbres et avec 100 estimateurs.**



 ----

In [ ]:
####### Complete this part ######## or die ####################
estimators = {
    "Tree": ...
    "Bagging": ...
}
###############################################################


# Loop over estimators to compare
for n, (name, estimator) in enumerate(estimators.items()):
    # Compute predictions
    y_predict = np.zeros((1000, n_repeat))

    for i in range(n_repeat):
        estimator.fit(X_train[i], y_train[i])
        y_predict[:, i] = estimator.predict(X_test)

    # Bias^2 + Variance + Noise decomposition of the mean squared error
    y_error = np.zeros(1000)

    for i in range(n_repeat):
        for j in range(n_repeat):
            y_error += (y_test[:, j] - y_predict[:, i]) ** 2

    y_error /= (n_repeat * n_repeat)

    y_noise = np.var(y_test, axis=1)
    y_bias = (f(X_test) - np.mean(y_predict, axis=1)) ** 2
    y_var = np.var(y_predict, axis=1)
    
    print("{0}\t: {1:.4f} (error) = {2:.4f} (bias^2) "
          " + {3:.4f} (var) + {4:.4f} (noise)".format(name,
                                                      np.mean(y_error),
                                                      np.mean(y_bias),
                                                      np.mean(y_var),
                                                      np.mean(y_noise)))


Vous devriez constater que là où le gain d'erreur est le plus important est la variance. C'est exactement ce que nous avions préalablement anticipé.

## IV. Boosting

L'idée première derrière la méthode de *boosting* est de construire des modèles "faibles" (potentiellement à peine meilleurs que le hasard) mais complémentaires entre eux possédant ainsi de bonnes performances en tant que groupe. Nous étudierons ici la méthode *AdaBoost* (i.e. Adaptive Boosting). Soit $\mathcal{X}\subseteq\mathbb{R}^d$ et $\mathcal{Y}=\{-1, 1\}$. On ne considèrera que le cas de la classification binaire bien que le propos se généralise à d'autres tâches de *machine learning*. Notre objectif est de construire un modèle final $h$ de $\mathcal{X}$ vers $\mathcal{Y}$ en s'appuyant sur un jeu de données $S_n$. Notons $w_i^j$ le poids associé à la donnée $(x_i, y_i)$ pour le modèle faible $j$. Initialisons les poids à $w_i^j=1/n$. Notre modèle consistera en $m$ classifieurs faibles.

*AdaBoost* fonctionne de la manière suivante :

1. On initialise notre compteur $j=1$ (i.e. on considère le premier modèle)
2. On optimise notre modèle sur le jeu de données : $\sum_i w_i^j\textbf{1}\{h_j(x_i)\neq y_i\}$

3.  On calcule l'erreur normalisée : $\epsilon_j=\frac{\sum_i w_i^j\textbf{1}\{h_j(x_i)\neq y_i\}}{\sum_i w_i^j}$ et on évalue l'importance du modèle courant pour notre meta-modèle : $\alpha_j=\text{ln}\Big(\frac{1-\epsilon_j}{\epsilon_j}\Big)$
4.  On met à jour la pondération des données pour le modèle suivant : $w_i^{j+1}=w_i^j\text{exp}\big(\alpha_j \textbf{1}\{h_j(x_i)\neq y_i\}\big)$
5.  Si $j<m$, on reprend à l'étape $2$ pour optimiser le modèle suivant.

Une fois les modèles optimisés, notre *meta-classifieur* permet de faire des prédictions de la manière suivante :

$$H(x)=\text{sign}\Big(\sum_j\alpha_j h_j(x)\Big)$$

Intuitivement, on cherche à favoriser les modèles qui "fonctionnent bien" et à donner de l'importance aux exemples d'apprentissage mal classés pour que les futurs classifieurs faibles se concentrent dessus.

----

**Pourquoi ces coefficients ?**


Considérons tout d'abord la *loss* exponentielle :

$$\ell(z)=\text{exp}\big(-z\big),$$

où $z=yh(x)$, $(x, y)\in S_n$. La *loss* est affichée juste après. De manière directe, on observe que si notre modèle prédit un score dont le signe est le même que celui du label, alors la *loss* est petite et, à l'inverse, si le signe est différent, notre *loss* sera grande.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x = np.linspace(-1, 1, 101)
y = np.exp(-x)

plt.figure(figsize=(12, 8))
plt.plot(x, y, label='Exponential loss')
y = (x<=0).astype(int)
x[51] = 0
plt.plot(x, y, label='0/1 loss')
plt.legend()
plt.show()

On cherche à construire un classifieur $H$ minimisant la *loss* exponentielle suivante :

$$\mathcal{L}(H)=\sum_{i=1}^n\ell(y_iH(x_i)),$$

où 

$$H(x)=\frac{1}{2}\sum_j\alpha_j h_j(x),$$

tels que $h_j$ sont nos classifieurs faibles et $\alpha_j$ les poids qui leur sont associés. Cependant, imaginons qu'au lieu de tout minimiser d'une seule fois, nous procédions itérativement, classifieur par classifieur. Ainsi, lorsqu'on optimise le classifieur $j$, tous les classifieurs $h_{1}, \ldots, h_{j-1}$ et leur poid $\alpha_1, \ldots, \alpha_{j-1}$ restent fixés. Notons $H_m(x)$ le classifieur total jusqu'au classifieur faible $h_m$. Notre *loss* se reformule ainsi :

$$\mathcal{L}_j(H)=\sum_{i=1}^n\text{exp}\Big(-y_iH_{j-1}(x_i)-\frac{1}{2}y_i\alpha_jh_j(x_i)\Big)=\sum_{i=1}^nw_i^j\text{exp}\Big(-\frac{1}{2}y_i\alpha_jh_j(x_i)\Big),$$

où $w_i^j = \text{exp}(-y_iH_{j-1}(x_i))$. Notons que si nous sommes bien entrain d'optimiser notre loss du point de vue de $h_j$ et $\alpha_j$, alors $w_i^j, \forall i$ sont des constantes. Notons $S_c^{m}$ l'ensemble des points correctement classés par $H_m$ et $S_i^m$ ceux qui à l'inverse ne le sont pas. Nous pouvons reformuler l'erreur précédente de la manière suivante :

$$\begin{aligned}
\mathcal{L}_j(H)&=e^{-\alpha_j/2}\sum_{i\in S_\mathcal{c}^j}w_i^m+e^{\alpha_j/2}\sum_{i\in S_\mathcal{i}^j}w_i^m\\
&=(e^{\alpha_j/2}-e^{-\alpha_j/2})\sum_{i=1}^nw_i^m\textbf{1}\{h_j(x_i)\neq y_i\}+e^{-\alpha_j/2}\sum_{i=1}^nw_i^m\\
&\propto (e^{\alpha_j/2}-e^{-\alpha_j/2})\sum_{i=1}^nw_i^m\textbf{1}\{h_j(x_i)\neq y_i\}
\end{aligned}$$

Du point de vu de l'optimisation de $h_j$, on remarque que cela revient à optimiser :

$$(e^{\alpha_j/2}-e^{-\alpha_j/2})\sum_{i=1}^nw_i^m\textbf{1}\{h_j(x_i)\neq y_i\},$$

où le choix de $\alpha_j$ n'a pas d'effet. Cela revient à réaliser l'étape $2$ de notre algorithme. Considérons maintenant l'optimisation de $\alpha_j$. En annulant la derivée en fonction de $\alpha_j$ on se rend compte que cela revient à le calculer comme indiqué à l'étape 3 ($\epsilon_j$ et $\alpha_j$).

----

Dans le cas de la régression, la stratégie va être de travailler sur les résidus des modèles précédents.

---
Un *stumps* est une fonction seuille. C'est une fonction qui prend une variable, et considère que toutes les données dont la valeur de cette variable est supériere (ou inférieure) à un seuil sont associées à la classe $1$ et les autres à la classe $-1$. Il s'agit tout simplement d'un arbre de profondeur $1$.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import zero_one_loss
from sklearn.ensemble import AdaBoostClassifier

**<span style='color:blue'> Exercice</span>** 
**Dans le code ci-dessous, complétez les trois sections. La première consiste à construire un *stump*, à le fiter et à calculer son taux d'erreur sur le test. La seconde consiste à calculer un arbre de décision de profondeur $9$ et de nombre minimum de point par feuille à $1$, à le fitter et à calculer son taux d'erreur sur le test. Enfin, la troisième partie consiste à construire un modèle AdaBoost où le classifieur faible est un *stump*.**



 ----

In [ ]:
X, y = datasets.make_hastie_10_2(n_samples=12000, random_state=1)

X_test, y_test = X[2000:], y[2000:]
X_train, y_train = X[:2000], y[:2000]

####### Complete this part ######## or die ####################
... stump instanciation
... fit
dt_stump_err = ...
###############################################################

####### Complete this part ######## or die ####################
... decision tree instanciation
... fit
dt_err = ...
###############################################################


####### Complete this part ######## or die ####################
n_estimators = 400
learning_rate = 1.

... AdaBoost instanciation
... fit
###############################################################

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111)

ax.plot([1, n_estimators], [dt_stump_err] * 2, 'k-',
        label='Decision Stump Error')
ax.plot([1, n_estimators], [dt_err] * 2, 'k--',
        label='Decision Tree Error')

ada_err = np.zeros((n_estimators,))
for i, y_pred in enumerate(ada.staged_predict(X_test)):
    ada_err[i] = zero_one_loss(y_pred, y_test)

ada_err_train = np.zeros((n_estimators,))
for i, y_pred in enumerate(ada.staged_predict(X_train)):
    ada_err_train[i] = zero_one_loss(y_pred, y_train)

ax.plot(np.arange(n_estimators) + 1, ada_err,
        label='AdaBoost Test Error',
        color='red')
ax.plot(np.arange(n_estimators) + 1, ada_err_train,
        label='AdaBoost Train Error',
        color='blue')

ax.set_ylim((0.0, 0.5))
ax.set_xlabel('Nombre d\'estimateurs')
ax.set_ylabel('Taux d\'erreur')

leg = ax.legend(loc='upper right', fancybox=True)
leg.get_frame().set_alpha(0.7)

plt.show()


## V. Les forêts aléatoires

Vous avez vu différentes manières d'agréger plusieurs classifieurs. Vous avez également étudié les arbres de décision. Un arbre pris individuellement, sans limite de profondeur possède une dimension VC infinie. Ou, dit autrement, il possède une forte capacité à sur-apprendre. Les *forêts aléatoires* ou *random forest* définissent une stratégie permettant d'agréger des arbres de décisions complémentaires. Chacun des arbres ne peut pas avoir été appris de la même manière puisque l'apprentissage est déterministe (on aurait ainsi $M$ arbres identiques et ça n'apporterait rien).

Il existe de multiples stratégies permettant de construire des arbres différents et nous en détaillons une ici. Il existe deux angles d'attaque que nous combinerons :
* Chaque arbre ne voit pas les mêmes données,
* Chaque arbre ne voit pas les mêmes *features* (chaque nœud voit des features choisis aléatoirement).

Une stratégie d'apprentissage est donc la suivante pour un des arbres :

1.  On construit un jeu de données $S^\prime$ en tirant uniformément des points de $S$ avec remise. Une fois que $|S^\prime|=n^\prime$, on s'arrête,
2. En supposant que nous ayons $d$ variables explicatives, on tire uniformément $d^\prime$ variables dans $d$,
3. On construit l'arbre avec le jeu de données $S^\prime$ et les $d^\prime$ variables sélectionnées (dans \texttt{sklearn} les variables sont choisies à chaque nœud).

Cette opération est répétée jusqu'à ce que tous nos arbres aient été construits. Ainsi en voyant moins de *features*, nos arbres sont plus stables et leur combinaisons ayant vu des informations différentes est elle-même plus stable.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import zero_one_loss
from sklearn.ensemble import RandomForestClassifier

**<span style='color:blue'> Exercice</span>** 
**En récupérant les résultats déjà calculés dans Boosting pour *stump* et *decision tree*, complétez le code suivant afin de construire et de fitter un *random forest* tel que la profondeur maximale de l'arbre soit $9$.**



 ----

In [ ]:
X, y = datasets.make_hastie_10_2(n_samples=12000, random_state=1)

X_test, y_test = X[2000:], y[2000:]
X_train, y_train = X[:2000], y[:2000]



learning_rate = 1.

max_n_estimators = 50

n_estimators_list = np.array(
    list(range(1, max_n_estimators + 1))
)

rf_err = np.zeros((n_estimators_list.shape[0],))
rf_err_train = np.zeros((n_estimators_list.shape[0],))

for j, nb_estimators in enumerate(n_estimators_list):
    ####### Complete this part ######## or die ####################
    ... random forest construction
    ... fit
    ###############################################################
    rf_err[j] = zero_one_loss(rf.predict(X_test), y_test)
    rf_err_train[j] = zero_one_loss(rf.predict(X_train), y_train)

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111)

ax.plot([1, max_n_estimators], [dt_stump_err] * 2, 'k-',
        label='Decision Stump Error')
ax.plot([1, max_n_estimators], [dt_err] * 2, 'k--',
        label='Decision Tree Error')

ax.plot(n_estimators_list, rf_err,
        label='RandomForest Test Error',
        color='red')

ax.plot(n_estimators_list, rf_err_train,
        label='RandomForest Train Error',
        color='blue')

ax.set_ylim((0.0, 0.5))
ax.set_xlabel('Nombre d\'estimateurs')
ax.set_ylabel('Taux d\'erreur')

leg = ax.legend(loc='upper right', fancybox=True)
leg.get_frame().set_alpha(0.7)

plt.show()
